# Experiments

We've now got our learning algorithm working, so let's run some simulations and see if these more "advanced" techniques really make much difference.  Recall we've got the following choices:

1. Different activation functions: sigmoid, hyperbolic tangent, or rectifier.
2. Different cost functions: MSE or CE.
3. Different initialization functions: uniform in $[-1,1]$ or Xavier.
4. Different learning rates

Let's investigate all of them.  We'll use the same architecture for each experiment -- 80+40+10 neurons.  The last layer will always be sigmoid neurons to force them into the appropriate range.  We'll run 10 epochs with a batch size of 50; this is enough epochs to make the difference known, but not so many that I'm going to spend an entire afternoon watching little numbers go down.

We'll investigate every combination of activation function, cost function, and initialization method.  For each one, we'll start with a learning rate of 1024, then cut it in half until it stops blowing up, and consider that the learning rate for the experiment.  Note that it's not fair to pick a uniform learning rate across all the experiments, since different rates work better for different activation functions and so on.

To start, let's write an experiment function which allows us to specify which parameters we want to use, and then run it.

In [1]:
from basic_nn import *
import time

def run_exp(act_fn, cost_fn, init_fn, learning_rate):
    np.random.seed(313) # for determinism
    
    # Step 1: pick architecture; in prose and parameters
    neuron_sizes = [80, 40]
    
    num_epochs = 10
    batch_size = 50
    
    # Step 2: initialize
    weights, biases = init_fn(n, k, neuron_sizes)
    acts = [act_fn for _ in range(0, len(weights))]
    acts[-1] = act_sigmoid # last one is always sigmoid
    
    # Step 3: train
    t1 = time.time()

    for epoch in range(0, num_epochs):
        # we'll keep track of the cost as we go
        total_cost = 0
        num_batches = 0

        for X_mb, Y_mb in get_mini_batches(batch_size, train_X, train_Y):
            x, z, y = forward_prop(weights, biases, acts, X_mb)

            bp_grad_w, bp_grad_b = back_prop(weights, biases, acts, cost_fn, X_mb, Y_mb, x, y, z)

            for i in range(0, len(weights)):
                weights[i] -= learning_rate * bp_grad_w[i] / len(X_mb)
                biases[i] -= learning_rate * bp_grad_b[i] / len(X_mb)

            total_cost += cost_fn(y[-1], Y_mb, aggregate=True)
            num_batches += 1

        cost = total_cost / num_batches # average cost
        print("Cost {2:0.7f} through epoch {0}; took {1:0.3f} seconds so far.".format(epoch, time.time()-t1, cost))
    
    # Step 4: evaluate
    _, _, y = forward_prop(weights, biases, acts, train_X)
    success_rate = classification_success_rate(y[-1], train_Y)
    print("After {0} epochs, got {1:0.3f}% classifications correct.".format(num_epochs, 100*success_rate))

# The Data

As before, we'll use the MNIST dataset.

In [2]:
from mnist_import import get_mnist_nice

train_X, train_Y, test_X, test_Y = get_mnist_nice()

n = train_X.shape[1]
k = train_Y.shape[1]

# Sigmoid Neurons

Let's get started!  We'll go from "worst" to "best," in terms of the reputation of the algorithm and its importance (in my opinion, based on what I've read and seen).  So we'll loop through initialization functions (least important), then cost functions (more important), then activation functions (most important).

I won't show the results of failed learning rates, but trust me that I've started at 1024 and halved until a good learning rate was found.

In [3]:
learning_rate = 4
run_exp(act_sigmoid, cost_MSE, initialize_network_uniform, learning_rate)

Cost 0.4591226 through epoch 0; took 4.522 seconds so far.
Cost 0.2402632 through epoch 1; took 9.972 seconds so far.
Cost 0.2083375 through epoch 2; took 13.626 seconds so far.
Cost 0.1922131 through epoch 3; took 17.168 seconds so far.
Cost 0.1811940 through epoch 4; took 20.715 seconds so far.
Cost 0.1733975 through epoch 5; took 24.251 seconds so far.
Cost 0.1668856 through epoch 6; took 27.767 seconds so far.
Cost 0.1617854 through epoch 7; took 31.340 seconds so far.
Cost 0.1574871 through epoch 8; took 34.895 seconds so far.
Cost 0.1534947 through epoch 9; took 38.408 seconds so far.
After 10 epochs, got 87.817% classifications correct.


In [4]:
learning_rate = 4
run_exp(act_sigmoid, cost_MSE, initialize_xavier_sigmoid, learning_rate)

Cost 0.3648476 through epoch 0; took 3.569 seconds so far.
Cost 0.1342702 through epoch 1; took 7.137 seconds so far.
Cost 0.1036181 through epoch 2; took 10.723 seconds so far.
Cost 0.0864062 through epoch 3; took 14.233 seconds so far.
Cost 0.0750066 through epoch 4; took 17.794 seconds so far.
Cost 0.0668906 through epoch 5; took 21.457 seconds so far.
Cost 0.0605494 through epoch 6; took 24.987 seconds so far.
Cost 0.0554018 through epoch 7; took 28.502 seconds so far.
Cost 0.0511987 through epoch 8; took 32.233 seconds so far.
Cost 0.0475507 through epoch 9; took 35.761 seconds so far.
After 10 epochs, got 97.738% classifications correct.


In [5]:
learning_rate = 4
run_exp(act_sigmoid, cost_CE, initialize_network_uniform, learning_rate)

Cost 1.1200734 through epoch 0; took 3.658 seconds so far.
Cost 0.4754880 through epoch 1; took 7.294 seconds so far.
Cost 0.3797369 through epoch 2; took 11.798 seconds so far.
Cost 0.3265063 through epoch 3; took 15.671 seconds so far.
Cost 0.2917096 through epoch 4; took 19.488 seconds so far.
Cost 0.2629701 through epoch 5; took 23.217 seconds so far.
Cost 0.2401513 through epoch 6; took 27.272 seconds so far.
Cost 0.2229482 through epoch 7; took 31.002 seconds so far.
Cost 0.2066669 through epoch 8; took 34.826 seconds so far.
Cost 0.1926037 through epoch 9; took 38.569 seconds so far.
After 10 epochs, got 98.055% classifications correct.


In [6]:
learning_rate = 4
run_exp(act_sigmoid, cost_CE, initialize_xavier_sigmoid, learning_rate)

Cost 0.9786934 through epoch 0; took 3.653 seconds so far.
Cost 0.3796509 through epoch 1; took 7.560 seconds so far.
Cost 0.2925477 through epoch 2; took 11.342 seconds so far.
Cost 0.2421980 through epoch 3; took 15.207 seconds so far.
Cost 0.2086297 through epoch 4; took 18.855 seconds so far.
Cost 0.1852873 through epoch 5; took 22.497 seconds so far.
Cost 0.1662026 through epoch 6; took 26.133 seconds so far.
Cost 0.1508255 through epoch 7; took 30.161 seconds so far.
Cost 0.1380346 through epoch 8; took 34.317 seconds so far.
Cost 0.1268596 through epoch 9; took 38.107 seconds so far.
After 10 epochs, got 98.882% classifications correct.


The results are quite impressive, in my opinion.  The initialization really matters -- observe that in both cases, switching from the naive uniform initialization to the Xavier method cut the error rate almost in half.  The networks that come from this method don't have the same saturation problems that a uniformly generated network would, and they train a lot more quickly.

Also notice the effect of having a proper cost function, which speeds training wonderfully.  Especially striking is in the case of a "bad" initialization; with the MSE cost function, the uniform network can only train to 88% accuraccy in 10 epochs, but in the same time, the CE cost function gets to 98% accuraccy.

# Hyperbolic Tangent Neurons

In [7]:
learning_rate = 2
run_exp(act_tanh, cost_MSE, initialize_network_uniform, learning_rate)

Cost 0.7630340 through epoch 0; took 4.774 seconds so far.
Cost 0.6679351 through epoch 1; took 8.960 seconds so far.
Cost 0.6529174 through epoch 2; took 15.267 seconds so far.
Cost 0.6452124 through epoch 3; took 19.580 seconds so far.
Cost 0.6405911 through epoch 4; took 23.502 seconds so far.
Cost 0.6367360 through epoch 5; took 27.533 seconds so far.
Cost 0.6339814 through epoch 6; took 31.500 seconds so far.
Cost 0.6313757 through epoch 7; took 35.274 seconds so far.
Cost 0.5807940 through epoch 8; took 39.690 seconds so far.
Cost 0.5258620 through epoch 9; took 44.160 seconds so far.
After 10 epochs, got 53.110% classifications correct.


In [8]:
learning_rate = 2
run_exp(act_tanh, cost_MSE, initialize_xavier_tanh, learning_rate)

Cost 0.2674983 through epoch 0; took 3.734 seconds so far.
Cost 0.0944498 through epoch 1; took 7.886 seconds so far.
Cost 0.0700270 through epoch 2; took 11.614 seconds so far.
Cost 0.0569199 through epoch 3; took 15.414 seconds so far.
Cost 0.0487330 through epoch 4; took 19.166 seconds so far.
Cost 0.0426666 through epoch 5; took 22.946 seconds so far.
Cost 0.0374035 through epoch 6; took 26.697 seconds so far.
Cost 0.0339940 through epoch 7; took 30.456 seconds so far.
Cost 0.0302312 through epoch 8; took 34.236 seconds so far.
Cost 0.0278998 through epoch 9; took 38.385 seconds so far.
After 10 epochs, got 98.767% classifications correct.


In [9]:
learning_rate = 2
run_exp(act_tanh, cost_CE, initialize_network_uniform, learning_rate)

Cost 1.7658294 through epoch 0; took 5.016 seconds so far.
Cost 0.6236970 through epoch 1; took 8.938 seconds so far.
Cost 0.5034897 through epoch 2; took 13.827 seconds so far.
Cost 0.4365804 through epoch 3; took 18.575 seconds so far.
Cost 0.3972372 through epoch 4; took 22.690 seconds so far.
Cost 0.3635418 through epoch 5; took 26.865 seconds so far.
Cost 0.3359065 through epoch 6; took 30.801 seconds so far.
Cost 0.3139082 through epoch 7; took 34.737 seconds so far.
Cost 0.2959597 through epoch 8; took 39.211 seconds so far.
Cost 0.2827874 through epoch 9; took 43.136 seconds so far.
After 10 epochs, got 96.712% classifications correct.


In [10]:
learning_rate = 2
run_exp(act_tanh, cost_CE, initialize_xavier_tanh, learning_rate)

Cost 0.7109427 through epoch 0; took 3.874 seconds so far.
Cost 0.3222831 through epoch 1; took 7.765 seconds so far.
Cost 0.2474860 through epoch 2; took 11.655 seconds so far.
Cost 0.2047924 through epoch 3; took 15.787 seconds so far.
Cost 0.1770856 through epoch 4; took 20.045 seconds so far.
Cost 0.1575822 through epoch 5; took 24.385 seconds so far.
Cost 0.1428242 through epoch 6; took 29.435 seconds so far.
Cost 0.1343289 through epoch 7; took 33.534 seconds so far.
Cost 0.1220170 through epoch 8; took 38.876 seconds so far.
Cost 0.1132335 through epoch 9; took 43.435 seconds so far.
After 10 epochs, got 98.818% classifications correct.


There is no real significance to the change in learning rates from 4 to 2; it is perhaps due to the steeper slope of hyperbolic tangent in general, as compared to the sigmoid function.

When I saw the first experiment, I was quite disappointed.  It looks like hyperbolic tangent was a huge under-performer.  But the second was impressive - with proper initialization, it dramatically outperformed the sigmoid network.  Similarly with the second pair.

The lesson, apparently, is that proper initialization is extremely important for hyperbolic tangent neurons, and doing it wrong results in bad, difficult-to-train networks which underperform compared to sigmoid neurons.  But with proper initialization, they outperform the sigmoid neurons (even when both are initialized properly).

The lesson about appropriate cost functions is the same as before, for the same reasons.  Note the lack of significance of this factor when the network is initialized properly, although this could be a coincidence.

# Rectifier Units

In [11]:
learning_rate = 1
run_exp(act_LeRU, cost_MSE, initialize_network_uniform, learning_rate)

Cost 1.0125692 through epoch 0; took 4.289 seconds so far.
Cost 0.9152028 through epoch 1; took 8.378 seconds so far.
Cost 0.8623041 through epoch 2; took 11.880 seconds so far.
Cost 0.8523041 through epoch 3; took 15.405 seconds so far.
Cost 0.8448071 through epoch 4; took 18.948 seconds so far.
Cost 0.8411744 through epoch 5; took 22.501 seconds so far.
Cost 0.8398791 through epoch 6; took 26.050 seconds so far.
Cost 0.8364738 through epoch 7; took 30.044 seconds so far.
Cost 0.8334717 through epoch 8; took 33.834 seconds so far.
Cost 0.8312291 through epoch 9; took 37.532 seconds so far.
After 10 epochs, got 28.380% classifications correct.


In [12]:
learning_rate = 1
run_exp(act_LeRU, cost_MSE, initialize_xavier_leru, learning_rate)

Cost 0.5230091 through epoch 0; took 3.548 seconds so far.
Cost 0.3698022 through epoch 1; took 7.261 seconds so far.
Cost 0.3511810 through epoch 2; took 10.822 seconds so far.
Cost 0.3409287 through epoch 3; took 14.332 seconds so far.
Cost 0.3345050 through epoch 4; took 18.772 seconds so far.
Cost 0.3298234 through epoch 5; took 22.898 seconds so far.
Cost 0.3268026 through epoch 6; took 27.196 seconds so far.
Cost 0.3241408 through epoch 7; took 30.893 seconds so far.
Cost 0.3214145 through epoch 8; took 34.834 seconds so far.
Cost 0.3194634 through epoch 9; took 38.354 seconds so far.
After 10 epochs, got 75.947% classifications correct.


In [13]:
learning_rate = 0.5
run_exp(act_LeRU, cost_CE, initialize_network_uniform, learning_rate)

Cost 31.8849991 through epoch 0; took 3.611 seconds so far.
Cost 29.4997479 through epoch 1; took 7.244 seconds so far.
Cost 29.3082271 through epoch 2; took 10.923 seconds so far.
Cost 29.1702642 through epoch 3; took 14.615 seconds so far.
Cost 29.0942340 through epoch 4; took 18.610 seconds so far.
Cost 29.0434740 through epoch 5; took 22.414 seconds so far.
Cost 28.9036678 through epoch 6; took 26.300 seconds so far.
Cost 28.7431739 through epoch 7; took 30.541 seconds so far.
Cost 28.4778167 through epoch 8; took 34.648 seconds so far.
Cost 28.2376060 through epoch 9; took 38.791 seconds so far.
After 10 epochs, got 31.007% classifications correct.


In [14]:
learning_rate = 1
run_exp(act_LeRU, cost_CE, initialize_xavier_leru, learning_rate)

Cost 0.7097867 through epoch 0; took 4.316 seconds so far.
Cost 0.2965490 through epoch 1; took 10.630 seconds so far.
Cost 0.2226534 through epoch 2; took 16.853 seconds so far.
Cost 0.1866233 through epoch 3; took 21.541 seconds so far.
Cost 0.1606866 through epoch 4; took 25.440 seconds so far.
Cost 0.1428151 through epoch 5; took 29.502 seconds so far.
Cost 0.1263069 through epoch 6; took 34.250 seconds so far.
Cost 0.1173063 through epoch 7; took 38.252 seconds so far.
Cost 0.1093971 through epoch 8; took 42.353 seconds so far.
Cost 0.1030406 through epoch 9; took 46.532 seconds so far.
After 10 epochs, got 99.073% classifications correct.


Interestingly LeRU units need a lower learning rate than the other options, and indeed have overflow problems quite often with ordinary-looking learning rates (e.g. 2).  This is perhaps because their slopes do not degrade like the sigmoid or hyperbolic tangent neurons do.

The final experiment slightly improved on hyperbolic tangent, but the first two experiments underperformed drastically compared to both sigmoid and hyperbolic tangent neurons.

The third experiment is left in as a cautionary tale.  Without regularization (which we will discuss soon), there is nothing stopping the coefficients from growing without bound. This is particularly troubling for rectifier units, where one can easily grow the input at an exponential rate across layers.  The `cost_CE` function is tolerant to sigmoid "overflows" to zero or one, but we still have major problems.  This initialization gives a massively over-activated network, and there's really nothing that can be done to fix it (at least, with the tools we've already discussed).

# In Defense of Rectifier Units

Supposedly, rectifier units are the new hotness, so why aren't they outperforming hyperbolic tangents, which are so 2006?  It turns out they really shine with a bigger network.  So let's do another experiment, where we give them more epochs and more neurons, do them both "right," and see which one really does better.

In [15]:
def run_bigger_exp(act_fn, cost_fn, init_fn, learning_rate):
    np.random.seed(313) # for determinism

    # Step 1: pick architecture; in prose and parameters
    num_epochs = 25
    batch_size = 50
    
    neuron_sizes = [100, 100]
    
    # Step 2: initialize
    weights, biases = init_fn(n, k, neuron_sizes)
    acts = [act_fn for _ in range(0, len(weights))]
    acts[-1] = act_sigmoid # last one is always sigmoid
    
    # Step 3: train
    t1 = time.time()

    for epoch in range(0, num_epochs):
        # we'll keep track of the cost as we go
        total_cost = 0
        num_batches = 0

        for X_mb, Y_mb in get_mini_batches(batch_size, train_X, train_Y):
            x, z, y = forward_prop(weights, biases, acts, X_mb)

            bp_grad_w, bp_grad_b = back_prop(weights, biases, acts, cost_fn, X_mb, Y_mb, x, y, z)

            for i in range(0, len(weights)):
                weights[i] -= learning_rate * bp_grad_w[i] / len(X_mb)
                biases[i] -= learning_rate * bp_grad_b[i] / len(X_mb)

            total_cost += cost_fn(y[-1], Y_mb, aggregate=True)
            num_batches += 1

        cost = total_cost / num_batches # average cost
        print("Cost {2:0.7f} through epoch {0}; took {1:0.3f} seconds so far.".format(epoch, time.time()-t1, cost))
    
    # Step 4: evaluate
    _, _, y = forward_prop(weights, biases, acts, train_X)
    success_rate = classification_success_rate(y[-1], train_Y)
    print("After {0} epochs, got {1:0.3f}% classifications correct.".format(num_epochs, 100*success_rate))

In [16]:
learning_rate = 2
run_bigger_exp(act_sigmoid, cost_CE, initialize_xavier_sigmoid, learning_rate)

Cost 1.7071007 through epoch 0; took 6.327 seconds so far.
Cost 0.4903770 through epoch 1; took 12.792 seconds so far.
Cost 0.3847985 through epoch 2; took 19.587 seconds so far.
Cost 0.3254435 through epoch 3; took 25.058 seconds so far.
Cost 0.2833317 through epoch 4; took 30.950 seconds so far.
Cost 0.2534444 through epoch 5; took 36.101 seconds so far.
Cost 0.2309697 through epoch 6; took 41.230 seconds so far.
Cost 0.2121886 through epoch 7; took 47.208 seconds so far.
Cost 0.1955491 through epoch 8; took 52.309 seconds so far.
Cost 0.1815544 through epoch 9; took 57.399 seconds so far.
Cost 0.1696140 through epoch 10; took 62.707 seconds so far.
Cost 0.1592080 through epoch 11; took 68.021 seconds so far.
Cost 0.1490028 through epoch 12; took 73.130 seconds so far.
Cost 0.1410483 through epoch 13; took 78.203 seconds so far.
Cost 0.1337619 through epoch 14; took 83.333 seconds so far.
Cost 0.1266575 through epoch 15; took 88.457 seconds so far.
Cost 0.1204249 through epoch 16; to

In [17]:
learning_rate = 2
run_bigger_exp(act_tanh, cost_CE, initialize_xavier_tanh, learning_rate)

Cost 0.6370673 through epoch 0; took 5.820 seconds so far.
Cost 0.2970022 through epoch 1; took 11.378 seconds so far.
Cost 0.2244473 through epoch 2; took 16.889 seconds so far.
Cost 0.1866230 through epoch 3; took 22.425 seconds so far.
Cost 0.1579253 through epoch 4; took 27.974 seconds so far.
Cost 0.1389525 through epoch 5; took 33.513 seconds so far.
Cost 0.1216503 through epoch 6; took 39.035 seconds so far.
Cost 0.1100904 through epoch 7; took 45.835 seconds so far.
Cost 0.1014891 through epoch 8; took 52.592 seconds so far.
Cost 0.0916942 through epoch 9; took 58.345 seconds so far.
Cost 0.0836763 through epoch 10; took 63.967 seconds so far.
Cost 0.0765112 through epoch 11; took 69.557 seconds so far.
Cost 0.0696678 through epoch 12; took 75.094 seconds so far.
Cost 0.0678068 through epoch 13; took 80.627 seconds so far.
Cost 0.0612133 through epoch 14; took 86.135 seconds so far.
Cost 0.0566286 through epoch 15; took 91.861 seconds so far.
Cost 0.0519932 through epoch 16; to

In [18]:
learning_rate = 1
run_bigger_exp(act_LeRU, cost_CE, initialize_xavier_leru, learning_rate)

Cost 0.6274595 through epoch 0; took 5.647 seconds so far.
Cost 0.2668945 through epoch 1; took 11.026 seconds so far.
Cost 0.2023016 through epoch 2; took 16.289 seconds so far.
Cost 0.1644078 through epoch 3; took 22.180 seconds so far.
Cost 0.1418413 through epoch 4; took 27.995 seconds so far.
Cost 0.1245527 through epoch 5; took 33.460 seconds so far.
Cost 0.1097182 through epoch 6; took 39.469 seconds so far.
Cost 0.0996820 through epoch 7; took 46.156 seconds so far.
Cost 0.0903848 through epoch 8; took 53.132 seconds so far.
Cost 0.0844181 through epoch 9; took 59.013 seconds so far.
Cost 0.0767940 through epoch 10; took 64.781 seconds so far.
Cost 0.0727951 through epoch 11; took 70.931 seconds so far.
Cost 0.0669383 through epoch 12; took 76.606 seconds so far.
Cost 0.0627852 through epoch 13; took 82.597 seconds so far.
Cost 0.0584317 through epoch 14; took 88.280 seconds so far.
Cost 0.0542786 through epoch 15; took 93.809 seconds so far.
Cost 0.0538788 through epoch 16; to

Well, as a result, we have a few observations:
1. With sufficient learning rate and epochs, and proper cost function and initialization, they all work well.
2. It is slightly true that tanh and LeRU beat sigmoid, in terms of cost and classification accuracy.
3. tanh and LeRU are still quite similar in terms of performance.
4. All the classification rates are quite astoundingly good.

Note that when the classification accuracy is high, you want to compare the relative errors in order to get a sense of your improvement.  So when the sigmoid got 99.333% correct, it got 0.667% wrong; similarly the hyperbolic tangent got 0.288% wrong, meaning that the sigmoid network made more than twice as many errors as the hyperbolic tangent network, and almost three times as many errors as the rectifier network.

It is point (4) that is most worth mentioning right now.  We will return to how to make networks learn faster (through momentum, or pre-training, or etc.) but at least for this example, the classification accuracy is incredible.  It turns out that we are *overfitting* badly, and need to introduce measures to prevent this.

It's bad practice to examine the test set too often, so as to avoid overfitting to the test set, but there's no harm in doing it occasionally, so long as we aren't modifying our algorithms too obviously to fit them.  Let's compare accuracy on the training and on the test set:

In [19]:
def run_test_exp(act_fn, cost_fn, init_fn, learning_rate):
    np.random.seed(313) # for determinism

    # Step 1: pick architecture; in prose and parameters
    num_epochs = 25
    batch_size = 50
    
    neuron_sizes = [100, 100]
    
    # Step 2: initialize
    weights, biases = init_fn(n, k, neuron_sizes)
    acts = [act_fn for _ in range(0, len(weights))]
    acts[-1] = act_sigmoid # last one is always sigmoid
    
    # Step 3: train
    t1 = time.time()

    for epoch in range(0, num_epochs):
        # we'll keep track of the cost as we go
        total_cost = 0
        num_batches = 0

        for X_mb, Y_mb in get_mini_batches(batch_size, train_X, train_Y):
            x, z, y = forward_prop(weights, biases, acts, X_mb)

            bp_grad_w, bp_grad_b = back_prop(weights, biases, acts, cost_fn, X_mb, Y_mb, x, y, z)

            for i in range(0, len(weights)):
                weights[i] -= learning_rate * bp_grad_w[i] / len(X_mb)
                biases[i] -= learning_rate * bp_grad_b[i] / len(X_mb)

            total_cost += cost_fn(y[-1], Y_mb, aggregate=True)
            num_batches += 1

        cost = total_cost / num_batches # average cost
    
    # Step 4: evaluate
    _, _, y = forward_prop(weights, biases, acts, train_X)
    success_rate = 100*classification_success_rate(y[-1], train_Y)
    print("After {1} epochs, got {0:0.3f}% classifications correct (training).".format(success_rate, num_epochs))
    
    _, _, y = forward_prop(weights, biases, acts, test_X)
    success_rate = 100*classification_success_rate(y[-1], test_Y)
    print("After {1} epochs got {0:0.3f}% classifications correct (test).".format(success_rate, num_epochs))

In [21]:
learning_rate = 2
run_test_exp(act_tanh, cost_CE, initialize_xavier_tanh, learning_rate)

After 25 epochs, got 99.712% classifications correct (training).
After 25 epochs got 97.970% classifications correct (test).


In [22]:
learning_rate = 2
run_test_exp(act_LeRU, cost_CE, initialize_xavier_leru, learning_rate)

After 25 epochs, got 99.717% classifications correct (training).
After 25 epochs got 97.920% classifications correct (test).


It's not that the classification accuracy is *bad* on the test set -- actually it's quite good.  But we won't be able to improve it by fitting the training set more closely, as we've nearly completely fit it.  We want to force our model to generalize better, without getting to look at the test set we need to generalize to.  The usual technique to accomplish this is to force the model to be *simpler*, in one sense or another, so that there isn't "space" to "memorize" the training set, and it has to actually learn.